In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from openpyxl.styles import numbers
import os

#Блок настроек отображения
pd.options.display.max_columns = None
pd.options.display.max_rows = 100

In [ ]:
data_upp = pd.read_excel("Ваш путь к выгрузке отчета")
data_ut = pd.read_excel("Ваш путь к выгрузке отчета")

## Первичная обработка данных

In [ ]:
#Очистка от погрешностей 1С
data_upp = data_upp.drop(index=[0, 1, 2, data_upp.shape[0]-1])
data_upp.drop(columns=['Unnamed: 1', 'Unnamed: 2', 'Unnamed: 5'], inplace=True)
data_upp = data_upp.rename(columns={'Unnamed: 0':'date', #Дата
                            'Unnamed: 3':'channel_sales', #Канал продаж
                            'Unnamed: 4':'partner', #Контрагент
                            'Unnamed: 6':'region', #Регион
                            'Unnamed: 7':'articul', #Артикул
                            'Unnamed: 8':'product', #Номенклатура
                            'Unnamed: 9':'category', #Категория товара
                            'Unnamed: 10':'group_product', #Группа товара
                            'Unnamed: 11':'subgroup_product', #Подгруппа товара
                            'Unnamed: 12':'volume', #Объем
                            'Unnamed: 13':'kit', #Набор поштучно
                            'Unnamed: 14':'status', #Статус
                            'Unnamed: 15':'start_sales', #Начало продаж
                            'Unnamed: 16':'status_new', #Новинка
                            'Unnamed: 17':'trade_m',#Трейд-маркетинг
                            'Unnamed: 18':'fact_cost', #Цена продажи (факт)
                            'Unnamed: 19':'calc_cost', #Цена прайс (расчетная)
                            'Unnamed: 20':'quantity', #Количество
                            'Unnamed: 21':'revenue_nds', #Выручка с НДС
                            'Unnamed: 22':'cost_without_discount', #Стоимость без скидок
                            'Unnamed: 23':'nds', #НДС
                            'Unnamed: 24':'prime_cost'}) #Производственная себестоимость 

data_ut = data_ut.drop(index=[0, 1, 2, 3, 4, 5, 6, data_ut.shape[0]-1])
data_ut.drop(columns=['Unnamed: 1', 'Unnamed: 2', 'Unnamed: 5', 'Unnamed: 12'], inplace=True)

data_ut = data_ut.rename(columns={'Unnamed: 0':'date', #Дата
                            'Unnamed: 3':'subdivision', #Подразделение
                            'Unnamed: 4':'Manager', #Менеджер
                            'Unnamed: 6':'category', #Категория товара (Общие)
                            'Unnamed: 7':'group_product', #Группа товара (Общие)
                            'Unnamed: 8':'subgroup_product', #Подгруппа товара (Общие)
                            'Unnamed: 9':'articul', #Артикул
                            'Unnamed: 10':'product', #Тип запасов или услуга
                            'Unnamed: 11':'stock_type', #Номенклатура
                            'Unnamed: 13':'quantity', #Количество
                            'Unnamed: 14':'revenue_nds', #Сумма выручки с НДС
                            'Unnamed: 15':'revenue', #Сумма выручки без НДС
                            'Unnamed: 16':'cost_nds', #Стоимость с НДС
                            'Unnamed: 17':'cost'}) #Стоимость без НДС

#Переработка форматов начального датасета
data_upp['date'] = pd.to_datetime(data_upp['date'], format='%d.%m.%Y %H:%M:%S')
data_upp['fact_cost'] = np.float64(data_upp['fact_cost'])
data_upp['calc_cost'] = np.float64(data_upp['calc_cost'])
data_upp['quantity'] = np.float64(data_upp['quantity'])
data_upp['revenue_nds'] = np.float64(data_upp['revenue_nds'])
data_upp['cost_without_discount'] = np.float64(data_upp['cost_without_discount'])
data_upp['nds'] = np.float64(data_upp['nds'])
data_upp['prime_cost'] = np.float64(data_upp['prime_cost'])

data_ut['date'] = pd.to_datetime(data_ut['date'], format='%d.%m.%Y %H:%M:%S')
data_ut['quantity'] = np.float64(data_ut['quantity'])
data_ut['revenue_nds'] = np.float64(data_ut['revenue_nds'])
data_ut['revenue'] = np.float64(data_ut['revenue'])
data_ut['cost_nds'] = np.float64(data_ut['cost_nds'])
data_ut['cost'] = np.float64(data_ut['cost'])

#Исправление старта продаж тех продуктов, с чьей датой выхода мы не определились
data_upp['start_sales'] = data_upp['start_sales'].replace('31.12.2999', 0)
data_upp['start_sales'] = pd.to_datetime(data_upp['start_sales'], format='mixed')

#Очистка данных УТ
data_ut = data_ut.query('not subgroup_product.isna()')
data_ut = data_ut.query('not category == "Услуги доставки"')
data_ut = data_ut.query('not stock_type == ""')

## Создания датасета для общего дашборда по выручке

In [ ]:
data_dashboard_upp = data_upp[['date', 'channel_sales', 'partner', 'region', 'product', 'category', 'group_product', 'kit', 'status', 'trade_m', 'revenue_nds']]

#Очистка от лишних данных
data_dashboard_upp = data_dashboard_upp.query('trade_m == "Нет" and kit!="Тестер"')
data_dashboard_upp.drop(columns=['trade_m', 'kit'], inplace=True)
data_dashboard_upp['partner'].replace({'Вайлдберриз ООО':'РВБ ООО'}, inplace=True)

#Перевод колонок в читабельный вид
data_dashboard_upp = data_dashboard_upp.rename(columns={
                            'date':'Дата', #Дата
                            'channel_sales':'Канал продаж', #Канал продаж
                            'partner':'Контрагент', #Контрагент
                            'region':'Регион', #Регион
                            'product':'Номенклатура', #Номенклатура
                            'category':'Категория товара', #Категория товара
                            'group_product':'Группа товара', #Группа товара
                            'status':'Статус', #Статус
                            'revenue_nds':'Выручка с НДС', #Выручка с НДС
                            })

#Выгрузка датасета
data_dashboard_upp.to_csv('for_dashboard_sales.csv')

## Создание датасета для дашборда по регионам

In [ ]:
data_dashboard_region = data_upp[['date', 'channel_sales', 'partner', 'region', 'product', 'category', 'group_product', 'kit', 'trade_m', 'revenue_nds', 'cost_without_discount']]
#Очистка от лишних данных
data_dashboard_region = data_dashboard_region.query('trade_m == "Нет" and kit!="Тестер"')
data_dashboard_region.drop(columns=['trade_m', 'kit'], inplace=True)
data_dashboard_region['partner'].replace({'Вайлдберриз ООО':'РВБ ООО'}, inplace=True)
#Рассчет поля размер скидки
data_dashboard_region['discount'] = 100 - (data_dashboard_region['revenue_nds'] / data_dashboard_region['cost_without_discount'] * 100)

#Оставляем канал продаж "Дистрибьюция"
data_dashboard_region = data_dashboard_region.query('channel_sales == "Дистрибуция"')
data_dashboard_region.drop(columns=['channel_sales'], inplace=True)

#Готовим данные из УТ
data_to_region = data_ut[['date', 'subdivision', 'product', 'category', 'group_product', 'revenue_nds']]

#Раскидываем ОАП по регионам
region_mapping = {'ИМ РнД': 'Ростовская область', 
                'ОАП Ростов': 'Ростовская область',
                'Розница РнД': 'Ростовская область',
                'Ростов-на-Дону': 'Ростовская область',
                'ОАП Краснодар': 'Краснодарский край',
                'ОАП Сочи':'Краснодарский край', 
                'Розница Сочи':'Краснодарский край',
                'ИМ Пермь':'Пермский край',
                'ОАП Пермь':'Пермский край',
                'Розница Пермь':'Пермский край',
                'Розница Иркутск':'Иркутская область',
                'ОАП Иркутск':'Иркутская область',
                'ИМ Иркутск':'Иркутская область',
                'ОАП Казань':'Республика Татарстан'
                }

data_to_region['region'] = data_to_region.subdivision.str.extract(r'(\w+\s+\w+)')[0].map(region_mapping)

#Рассчитываем метрики скидки
data_to_region['discount'] = 40
data_to_region = data_to_region.rename(columns={'revenue_nds':'cost_without_discount',
                                                'subdivision':'partner'})
data_to_region['revenue_nds'] = data_to_region['cost_without_discount'] * (100 - data_to_region['discount']) / 100

#Выстраиваем датасет
data_to_region = data_to_region[['date', 'partner', 'region', 'product', 'category', 'group_product', 'revenue_nds', 'cost_without_discount', 'discount']]

#Объединение и экспорт
data_dashboard_region = pd.concat([data_dashboard_region, data_to_region], ignore_index=True)

#Перевод колонок в читабельный вид
data_dashboard_region = data_dashboard_region.rename(columns={
                            'date':'Дата', #Дата
                            'partner':'Контрагент', #Контрагент
                            'region':'Регион', #Регион
                            'product':'Номенклатура', #Номенклатура
                            'category':'Категория товара', #Категория товара
                            'group_product':'Группа товара', #Группа товара
                            'revenue_nds':'Выручка с НДС', #Выручка с НДС
                            'cost_without_discount':'Стоимость без скидок', #Стоимость без скидок
                            'discount':'Скидка в %' #Размер скидки
                            })

data_dashboard_region.to_csv('for_dashboard_region.csv')

## Создание датасета для дашборда по товарам

In [ ]:
data_dashboard_product = data_upp[['date', 'channel_sales', 'partner', 'region', 'product', 'category', 'group_product', 'kit', 'trade_m', 'revenue_nds']]
#Очистка от лишних данных
data_dashboard_product = data_dashboard_product.query('trade_m == "Нет" and kit!="Тестер"')
data_dashboard_product.drop(columns=['trade_m', 'kit'], inplace=True)
data_dashboard_product['partner'].replace({'Вайлдберриз ООО':'РВБ ООО'}, inplace=True)

#Перевод колонок в читабельный вид
data_dashboard_product = data_dashboard_product.rename(columns={
                            'date':'Дата', #Дата
                            'channel_sales':'Канал продаж', #Канал продаж
                            'partner':'Контрагент', #Контрагент
                            'region':'Регион', #Регион
                            'product':'Номенклатура', #Номенклатура
                            'category':'Категория товара', #Категория товара
                            'group_product':'Группа товара', #Группа товара
                            'revenue_nds':'Выручка с НДС', #Выручка с НДС
                            })

data_dashboard_product.to_csv('for_dashboard_product.csv')

## Выгрузка предобработанных общих таблиц в Excel

In [ ]:
#Перевод колонок в читабельный вид
data_upp = data_upp.rename(columns={'date':'Дата', #Дата
                            'channel_sales':'Канал продаж', #Канал продаж
                            'partner':'Контрагент', #Контрагент
                            'region':'Регион', #Регион
                            'articul':'Артикул', #Артикул
                            'product':'Номенклатура', #Номенклатура
                            'category':'Категория товара', #Категория товара
                            'group_product':'Группа товара', #Группа товара
                            'subgroup_product':'Подгруппа товара', #Подгруппа товара
                            'volume':'Объем', #Объем
                            'kit':'Набор поштучно', #Набор поштучно
                            'status':'Статус', #Статус
                            'start_sales':'Начало продаж', #Начало продаж
                            'status_new':'Новинка', #Новинка
                            'fact_cost':'Цена продажи (факт)', #Цена продажи (факт)
                            'calc_cost':'Цена прайс (расчетная)', #Цена прайс (расчетная)
                            'quantity':'Количество', #Количество
                            'revenue_nds':'Выручка с НДС', #Выручка с НДС
                            'cost_without_discount':'Стоимость без скидок', #Стоимость без скидок
                            'nds':'НДС', #НДС
                            'prime_cost':'Производственная себестоимость '}) #Производственная себестоимость 

data_ut = data_ut.rename(columns={'date':'Дата', #Дата
                            'subdivision':'Подразделение', #Контрагент
                            'Manager':'Менеджер', 
                            'category':'Категория товара (Общие)', #Категория товара
                            'group_product':'Группа товара (Общие)', #Группа товара
                            'subgroup_product':'Подгруппа товара (Общие)', #Подгруппа товара
                            'articul':'Артикул', #Артикул
                            'stock_type':'Тип запасов или услуга', #Тип запасов или услуга
                            'product':'Номенклатура', #Номенклатура
                            'quantity':'Количество', #Количество
                            'revenue_nds':'Сумма выручки с НДС', #Выручка с НДС
                            'revenue':'Сумма выручки без НДС', #Выручка без НДС
                            'cost_nds':'Стоимость с НДС', #Стоимость с НДС
                            'cost':'Стоимость без НДС'}) #Стоимость без НДС

data_upp.to_excel('УПП_обработанный.xlsx')
data_ut.to_excel('УТ_обработанный.xlsx')

## Если необходимо обработать несколько выгрузок и совместить в один датасет

In [ ]:
folder_path = 'Путь к папке с выгрузками'
all_data = []  # Список для хранения DataFrame

for file in os.listdir(folder_path):
    if file.endswith('.xlsx'):
        df = pd.read_excel(os.path.join(folder_path, file))
        df.columns = df.iloc[2]
        df = df.iloc[3:]
        df = df.drop(columns=df.columns[df.columns.isna()]) 
        all_data.append(df)

# Объединяем все DataFrame в один
combined_df = pd.concat(all_data, ignore_index=True)

combined_df['По дням'] = pd.to_datetime(
    combined_df['По дням'], 
    format='%d.%m.%Y', 
    errors='coerce'
)

numeric_cols = ['Неделя года', 'Месяц года', 'Квартал года', 'Год']
for col in numeric_cols:
    combined_df[col] = pd.to_numeric(combined_df[col], errors='coerce').astype('Int64')
    
combined_df

## Сохранение в Excel с правильными форматами

In [ ]:
output_path = os.path.join(folder_path, 'Объединенный_отчет.xlsx')

with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
    combined_df.to_excel(writer, index=False, sheet_name='Данные')
    
    # Получаем объекты для форматирования
    workbook = writer.book
    worksheet = writer.sheets['Данные']
    
    # Формат даты (дд.мм.гггг)
    date_format = numbers.BUILTIN_FORMATS[14]  # 14 - код формата даты
    worksheet.column_dimensions['A'].width = 12  # Ширина столбца
    for cell in worksheet['A']:
        if cell.row > 1:  # Пропускаем заголовок
            cell.number_format = date_format
    
    # Формат целых чисел
    int_format = numbers.BUILTIN_FORMATS[1]  # 0 без десятичных
    for col in numeric_cols:
        col_letter = chr(ord('B') + numeric_cols.index(col))  # B, C, D, E
        worksheet.column_dimensions[col_letter].width = 12
        for cell in worksheet[col_letter]:
            if cell.row > 1:
                cell.number_format = int_format

print(f'Все файлы обработаны и объединены в {output_path}')